# ABS Monthly Consumer Price Index Indicator 6484

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_abs_meta_and_data,
    get_plot_constants,
    find_id,
    get_meta_constants,
)
from plotting import (
    finalise_plot,
    set_chart_dir,
    clear_chart_dir,
    plot_growth,
    line_plot,
    calc_growth,
    COLOR_BLUE,
    COLOR_AMBER,
    COLOR_RED,
)
from henderson import hma

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
catalogue_id = "6484"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_dict = get_abs_meta_and_data(catalogue_id, verbose=False)
if abs_dict is None:
    sys.exit(-1)
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col, tdesc_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Source modification date: 2023-11-29 00:43:55+00:00
Retrieving data from the cache file: ABS_CACHE/b1aa54df40869bbe37b234e3d7acd09f--All-Time-Series-Spreadsheets.zip
Extracting DataFrames from the zip-file ...


## Plot

### Set-up 

In [3]:
plt.style.use("fivethirtyeight")
set_chart_dir(CHART_DIR)
clear_chart_dir(CHART_DIR)

TAREGT_COLOR = "#dddddd"

### Supporting functions

In [4]:
def annualise_rates(series: pd.Series, periods: int = 12) -> pd.Series:
    """Annualise a growth rate for a period.
    Note: returns a percentage (and not a rate)!"""

    return (((1 + series) ** periods) - 1) * 100

### Plot growth - combined line/bar charts

In [5]:
def cpi_growth():
    rows = meta[meta[did_col].str.contains("Index Numbers")]

    for index, row in rows.iterrows():
        # get data for the plot
        did, table = row[did_col], row[table_col]
        series_type, series_id = row[type_col], row[id_col]
        name = did.split(";")[1].strip()
        index = abs_dict[table][series_id]
        growth = calc_growth(index)

        # print the last few values
        if False:
            print(name)
            print(pd.DataFrame(growth, index=["Annual", "Monthly"]).T.tail().T)

        # plot the data
        ax = plot_growth(*growth)
        if ax:
            ax.axhspan(
                2, 3, color=TAREGT_COLOR, zorder=-1, label="Annual inflation target"
            )
            ax.legend(loc="best", fontsize="small")
            LONG_NAME = 40
            separator = " " if len(name) < LONG_NAME else "\n"
            finalise_plot(
                ax,
                title=f"Price Growth:{separator}{name}",
                ylabel="Per cent",
                rfooter=source,
                lfooter=f"Australia: {series_type.capitalize()} series. ",
                show=SHOW,
            )

In [6]:
cpi_growth()

### Annualised growth

In [7]:
def line_and_target_plot(series: pd.Series, **kwargs) -> None:
    ax = series.plot(lw=2, color=COLOR_RED)
    ax.axhspan(2, 3, color=TAREGT_COLOR, zorder=-1, label="Annual inflation target")
    finalise_plot(ax, **kwargs)


def annualised_growth() -> None:
    SMOOTHING = 13

    for series_name in (
        "Index Numbers ;  All groups CPI ;  Australia ;",
        "Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;",
    ):
        print(series_name)
        table = "1"
        selector = {series_name: did_col}
        series_id, units = find_id(meta, selector, verbose=False)
        series_type = (
            "Seasonally Adjusted"
            if "seasonally adjusted" in series_name
            else "Original"
        )
        name = series_name.split(";")[1].strip()
        LONG_NAME = 20
        separator = " " if len(name) < LONG_NAME else "\n"
        name = f"{separator}{name}"
        lfooter = f"Australia. {series_type.capitalize()} series. "

        quarterly = annualise_rates(
            abs_dict[table][series_id].pct_change(3), 4
        ).dropna()
        print(f'Quarterly annualised endpoint: {quarterly.iloc[-1]:0.2f}%')
        line_and_target_plot(
            quarterly,
            title=f"Quarterly Price Growth Annualised:{name}",
            ylabel="Per cent",
            lfooter=lfooter,
            rfooter=source,
            show=SHOW,
        )

        quarterly_s = hma(quarterly, SMOOTHING)
        print(f'Quarterly annualised & smoothed endpoint: {quarterly_s.iloc[-1]:0.2f}%')
        line_and_target_plot(
            quarterly_s,
            title=f"Quarterly Price Growth Annualised/Smoothed:{name}",
            ylabel="Per cent",
            lfooter=f"{lfooter}{SMOOTHING}-term Henderson moving average. ",
            rfooter=source,
            show=SHOW,
        )

        monthly = annualise_rates(abs_dict[table][series_id].pct_change(1), 12).dropna()
        print(f'Monthly annualised endpoint: {monthly.iloc[-1]:0.2f}%')
        line_and_target_plot(
            monthly,
            title=f"Monthly Price Growth Annualised:{name}",
            ylabel="Per cent",
            lfooter=lfooter,
            rfooter=source,
            show=SHOW,
        )

        monthly_s = hma(monthly, SMOOTHING)
        print(f'Monthly annualised & smoothed endpoint: {monthly_s.iloc[-1]:0.2f}%')
        line_and_target_plot(
            monthly_s,
            title=f"Monthly Price Growth Annualised/Smoothed:{name}",
            ylabel="Per cent",
            lfooter=f"{lfooter}{SMOOTHING}-term Henderson moving average. ",
            rfooter=source,
            show=SHOW,
        )
        print()

In [8]:
annualised_growth()

Index Numbers ;  All groups CPI ;  Australia ;
Quarterly annualised endpoint: 3.38%
Quarterly annualised & smoothed endpoint: 4.58%
Monthly annualised endpoint: -3.88%
Monthly annualised & smoothed endpoint: 2.65%

Index Numbers ;  All groups CPI, seasonally adjusted ;  Australia ;
Quarterly annualised endpoint: 3.03%
Quarterly annualised & smoothed endpoint: 4.74%
Monthly annualised endpoint: -4.83%
Monthly annualised & smoothed endpoint: 2.10%



### Selected sub items

In [9]:
annual = "Corresponding Month of Previous Year"
selectors = {
    "Food and non-alcoholic beverages": {
        "Food and non-alcoholic beverages": did_col,
        "Original": type_col,
        annual: did_col,
    },
    "Automotive fuel": {
        "Automotive fuel": did_col,
        "Original": type_col,
        annual: did_col,
    },
}

data = pd.DataFrame()
for name, selector in selectors.items():
    id, units = find_id(meta, selector, verbose=False)
    table = meta.loc[meta["Series ID"] == id, "Table"].values[0]
    data[name] = abs_dict[table][id]

title = "Monthly CPI Indicator - Annual Growth - Food & Fuel"
tag = "1"
series_type = "Original series"
line_plot(
    data,
    title=title,
    ylabel="Per cent",
    color=[COLOR_BLUE, COLOR_AMBER],
    width=1.5,
    tags=tag,
    legend={"loc": "best", "fontsize": "x-small"},
    rfooter=f"{source}",
    lfooter=f"Australia: {series_type}",
    show=SHOW,
)

In [10]:
selectors = {
    "Rents": {
        "Rents": did_col,
        "Original": type_col,
        annual: did_col,
    },
    "New dwelling purchase by owner-occupiers": {
        "New dwelling purchase by owner-occupiers": did_col,
        "Original": type_col,
        annual: did_col,
    },
}

data = pd.DataFrame()
for name, selector in selectors.items():
    id, units = find_id(meta, selector, verbose=False)
    table = meta.loc[meta["Series ID"] == id, "Table"].values[0]
    data[name] = abs_dict[table][id]

title = "Monthly CPI Indicator - Annual Growth - Residential"
tag = "2"
series_type = "Original series"
line_plot(
    data,
    title=title,
    ylabel="Per cent",
    color=[COLOR_BLUE, COLOR_AMBER],
    width=1.5,
    tags=tag,
    legend={"loc": "best", "fontsize": "x-small"},
    rfooter=f"{source}",
    lfooter=f"Australia: {series_type}",
    show=SHOW,
)

## Finished

In [11]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Thu Dec 21 2023 20:46:40

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.18.1

sys       : 3.11.7 | packaged by conda-forge | (main, Dec 15 2023, 08:41:36) [Clang 16.0.6 ]
pandas    : 2.1.4
matplotlib: 3.8.2

Watermark: 2.4.3



In [12]:
print("Done")

Done
